## Sintonia de Parâmetros SIFT

In [ ]:
import pano
import cv2
import numpy as np
import timeit
import matplotlib.pyplot as plt
import os
import sys

dir1 = '../Conjunto1'
dir2 = '../Conjunto2'
dir3 = '../Conjunto3'
dir4 = '../Conjunto4'

conjunto1 = pano.ler_conjunto_imagens(dir1)
conjunto2 = pano.ler_conjunto_imagens(dir2)
conjunto3 = pano.ler_conjunto_imagens(dir3)
conjunto4 = pano.ler_conjunto_imagens(dir4)

In [ ]:
def sintonizar(imagem1, imagem2, num_features = 0, niveis_oitava = 3, 
                  threshold_contraste = 0.04, threshold_borda = 10, sigma = 1.6,
                  ratio_threshold = 0.5):   

    # Calcula keypoints e descritores utilizando SIFT
    t_1 = timeit.default_timer()
    keypoints_imagem1, descritores_imagem1 = pano.executar_SIFT(imagem1, num_features, niveis_oitava, threshold_contraste, 
                  threshold_borda, sigma)
    
    keypoints_imagem2, descritores_imagem2 = pano.executar_SIFT(imagem2, num_features, niveis_oitava, threshold_contraste, 
                  threshold_borda, sigma)
    t_2 = timeit.default_timer()

    # Plotar keypoints das duas imagens
    plot1 = cv2.drawKeypoints(imagem1, keypoints_imagem1, 0, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plot2 = cv2.drawKeypoints(imagem2, keypoints_imagem2, 0, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plot_img = np.concatenate((plot1, plot2), axis=1)
    print("Imprimindo keypoints...")
    pano.imprimir_imagem(plot_img)
    print("Quantidade de keypoints imagem 1:", len(keypoints_imagem1))
    print("Quantidade de keypoints imagem 2:", len(keypoints_imagem2))

    # Realiza o casamento entre keypoints das imagens
    t_3 = timeit.default_timer()
    casamentos = pano.casar_imagens(descritores_imagem1, descritores_imagem2,ratio_threshold=ratio_threshold)
    t_4 = timeit.default_timer()

    # Plotar casamento das duas imagens
    plot_casamentos = cv2.drawMatches(imagem1, keypoints_imagem1, imagem2, keypoints_imagem2, casamentos[:100], None,
                                      matchColor=(255, 0, 0),  # draw matches in green color
                                      singlePointColor=None, flags=2)
    print("\nImprimindo casamentos...")
    pano.imprimir_imagem(plot_casamentos)

    tempo = (t_4 - t_3) + (t_2 - t_1)

    print("Quantidade de casamentos:", len(casamentos))
    print("Tempo de exeucação: {:.2f} segundos\n".format(tempo))

    return len(keypoints_imagem1), len(keypoints_imagem2), len(casamentos), tempo

## Sigma

In [ ]:
sigmas = np.linspace(0.8, 24, 10)
sigmas = np.round(sigmas, 2)
kp1 = []
kp2 = []
m = []
temp = []

for sigma in sigmas:
  print("Sintonizando com sigma =", sigma)
  a,b,c,d = sintonizar(conjunto1[1], conjunto1[2], sigma = sigma)
  kp1.append(a)
  kp2.append(b)
  m.append(c)
  temp.append(d)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].plot(sigmas, kp1,  '-o', color='g')
axs[0, 0].set_ylabel("Quantidade de keypoints imagem 1")
axs[1, 1].set_xlabel("Sigma")
axs[0, 1].plot(sigmas, kp2,  '-o', color='b')
axs[0, 1].set_ylabel("Quantidade de keypoints imagem 2")
axs[1, 1].set_xlabel("Sigma")
axs[1, 0].plot(sigmas, temp, '-o', color='orange')
axs[1, 0].set_ylabel("Tempo de execução")
axs[1, 1].set_xlabel("Sigma")
axs[1, 1].plot(sigmas, m,    '-o', color='r')
axs[1, 1].set_ylabel("Quantidade de casamentos")
axs[1, 1].set_xlabel("Sigma")
plt.show()

## Níveis de Oitava

In [ ]:
niveis = np.linspace(1, 10, 10, dtype=int)
kp1 = []
kp2 = []
m = []
temp = []

for nivel in niveis:
  print("Sintonizando com níveis de oitava =", nivel)
  a,b,c,d = sintonizar(conjunto1[1], conjunto1[2], niveis_oitava = nivel)
  kp1.append(a)
  kp2.append(b)
  m.append(c)
  temp.append(d)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].plot(niveis, kp1,  '-o', color='g')
axs[0, 0].set_ylabel("Quantidade de keypoints imagem 1")
axs[1, 1].set_xlabel("Quantidade de níveis")
axs[0, 1].plot(niveis, kp2,  '-o', color='b')
axs[0, 1].set_ylabel("Quantidade de keypoints imagem 2")
axs[1, 1].set_xlabel("Quantidade de níveis")
axs[1, 0].plot(niveis, temp, '-o', color='orange')
axs[1, 0].set_ylabel("Tempo de execução")
axs[1, 1].set_xlabel("Quantidade de níveis")
axs[1, 1].plot(niveis, m,    '-o', color='r')
axs[1, 1].set_ylabel("Quantidade de casamentos")
axs[1, 1].set_xlabel("Quantidade de níveis")
plt.show()

## Threshold de Contraste

In [ ]:
contrastes = np.linspace(0.01, 0.2, 10)
contrastes = np.round(contrastes, 2)
kp1 = []
kp2 = []
m = []
temp = []

for contraste in contrastes:
  print("Sintonizando com threshold de contraste =", contraste)
  a,b,c,d = sintonizar(conjunto1[1], conjunto1[2], threshold_contraste = contraste)
  kp1.append(a)
  kp2.append(b)
  m.append(c)
  temp.append(d)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].plot(contrastes, kp1,  '-o', color='g', label='Blue stars')
axs[0, 0].set_ylabel("Quantidade de keypoints imagem 1")
axs[1, 1].set_xlabel("Threshold de contraste")
axs[0, 1].plot(contrastes, kp2,  '-o', color='b')
axs[0, 1].set_ylabel("Quantidade de keypoints imagem 2")
axs[1, 1].set_xlabel("Threshold de contraste")
axs[1, 0].plot(contrastes, temp, '-o', color='orange')
axs[1, 0].set_ylabel("Tempo de execução")
axs[1, 1].set_xlabel("Threshold de contraste")
axs[1, 1].plot(contrastes, m,    '-o', color='r')
axs[1, 1].set_ylabel("Quantidade de casamentos")
axs[1, 1].set_xlabel("Threshold de contraste")
plt.show()

## Threshold de Borda

In [ ]:
bordas = np.linspace(1, 20, 10)
bordas = np.round(bordas, 2)
kp1 = []
kp2 = []
m = []
temp = []

for borda in bordas:
  print("Sintonizando com threshold de borda =", borda)
  a,b,c,d = sintonizar(conjunto1[1], conjunto1[2], threshold_borda = borda)
  kp1.append(a)
  kp2.append(b)
  m.append(c)
  temp.append(d)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].plot(bordas, kp1,  '-o', color='g')
axs[0, 0].set_ylabel("Quantidade de keypoints imagem 1")
axs[0, 0].set_xlabel("Threshold de borda")
axs[0, 1].plot(bordas, kp2,  '-o', color='b')
axs[0, 1].set_ylabel("Quantidade de keypoints imagem 2")
axs[0, 1].set_xlabel("Threshold de borda")
axs[1, 0].plot(bordas, temp, '-o', color='orange')
axs[1, 0].set_ylabel("Tempo de execução")
axs[1, 0].set_xlabel("Threshold de borda")
axs[1, 1].plot(bordas, m,    '-o', color='r')
axs[1, 1].set_ylabel("Quantidade de casamentos")
axs[1, 1].set_xlabel("Threshold de borda")
plt.show()

## Número de Features

In [ ]:
feats = [10,20,30,40,50,60,70,80,90,100]
kp1 = []
kp2 = []
m = []
temp = []

for feat in feats:
  print("Sintonizando com número de features =", feat)
  a,b,c,d = sintonizar(conjunto1[1], conjunto1[2], num_features = feat, niveis_oitava = 3, 
                  threshold_contraste = 0.04, threshold_borda = 10, sigma = 1.6,
                  ratio_threshold = 1)
  kp1.append(a)
  kp2.append(b)
  m.append(c)
  temp.append(d)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].plot(feats, kp1,  '-o', color='g')
axs[0, 0].set_ylabel("Quantidade de keypoints imagem 1")
axs[0, 0].set_xlabel("Número de features")
axs[0, 1].plot(feats, kp2,  '-o', color='b')
axs[0, 1].set_ylabel("Quantidade de keypoints imagem 2")
axs[0, 1].set_xlabel("Número de features")
axs[1, 0].plot(feats, temp, '-o', color='orange')
axs[1, 0].set_ylabel("Tempo de execução")
axs[1, 0].set_xlabel("Número de features")
axs[1, 1].plot(feats, m,    '-o', color='r')
axs[1, 1].set_ylabel("Quantidade de casamentos")
axs[1, 1].set_xlabel("Número de features")
plt.show()

## Ratio Threshold

In [ ]:
ratios = np.linspace(0.0, 1.0, 10)
ratios = np.round(ratios, 2)
kp1 = []
kp2 = []
m = []
temp = []

for ratio in ratios:
  print("Sintonizando com ratio threshold =", ratio)
  a,b,c,d = sintonizar(conjunto1[1], conjunto1[2], ratio_threshold = ratio)
  kp1.append(a)
  kp2.append(b)
  m.append(c)
  temp.append(d)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
axs[0, 0].plot(ratios, kp1,  '-o', color='g')
axs[0, 0].set_ylabel("Quantidade de keypoints imagem 1")
axs[0, 0].set_xlabel("Ratio Threshold")
axs[0, 1].plot(ratios, kp2,  '-o', color='b')
axs[0, 1].set_ylabel("Quantidade de keypoints imagem 2")
axs[0, 1].set_xlabel("Ratio")
axs[1, 0].plot(ratios, temp, '-o', color='orange')
axs[1, 0].set_ylabel("Tempo de execução")
axs[1, 0].set_xlabel("Ratio")
axs[1, 1].plot(ratios, m,    '-o', color='r')
axs[1, 1].set_ylabel("Quantidade de casamentos")
axs[1, 1].set_xlabel("Ratio")
plt.show()